# Gaussian Processes
## Hands-On Code Tutorial
*   Instructor: Joel Paulson, Department of Chemical and Biomolecular Engineering, The Ohio State University
*   Location: Sargent Centre Summer School on Bayesian Optimization
*   Date: 09/03/24

# Notebook Overview

This notebook provides an introduction to Gaussian process (GP) modeling in Python, using mostly the flexible and scalable Python package [GPyTorch](https://gpytorch.ai/). It is broken up into the following 4 major sections:
0. A Quick PyTorch Primer.
  * This section is meant to provide a short introduction to key features of PyTorch that we will need throughout the sections.
1. GPs from Scratch:
  * In this section, we will implement the core components of a Gaussian process from scratch including defining a kernel, drawing GP samples from the prior and posterior, and estimating hyperparamters using the marginal log likelihood function.
2. Introduction to GPyTorch
  * This section provides a high-level introduction to the GPyTorch package such that you can see how to go about using it to perform basic regression tasks (with multiple input features). It also includes an exercise where you need to define your own custom kernel for the case of Brownian motion.
3. GP Regression: Modeling CO$_2$ Levels at the Mauna Loa Observatory
  * In this section, you will have the chance to apply the different things you have learned in the previous sections to model the monthly average atmospheric CO$_2$ concentration collected at the Mauna Loa Observatory in Hawaii. This example is based on Section 5.4.3 of the [Gaussian Processes for Machine Learning book](https://gaussianprocess.org/gpml/chapters/RW.pdf) and is interesting as it requires relatively complex kernel engineering to achieve good test (and extrapolation) performance in practice.

# Setup (Install Packages)

Not all the packages that we plan to use come installed in Google Colab, so run the code below to install external packages. Note that the ```&> /dev/null``` part of the code just supresses the output of the bash command.

In [ ]:
!pip install gpytorch &> /dev/null  #add gpytorch (https://gpytorch.ai/)
!pip install botorch  &> /dev/null  #add botorch (https://botorch.org/)

In [ ]:
# import all the necessary packages used throughout this notebook
from botorch import fit_gpytorch_mll
import gpytorch
from gpytorch.constraints import Positive
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import scipy.stats as stats
from sklearn.datasets import fetch_openml
import torch

# 0. A Quick PyTorch Primer

PyTorch is an open-source deep learning framework that provides a flexible and intuitive interface for building and training neural networks. It is particularly well-suited for tensor manipulation and automatic differentiation, making it a popular choice among researchers, developers, and practitioners for a wide range of machine learning tasks.

## 0.1 Key Features
* **Tensor Manipulation:** PyTorch tensors are multi-dimensional arrays similar to NumPy arrays, but with additional capabilities for GPU acceleration. They form the building blocks for all computations in PyTorch and can be used for all sorts of data manipulation. Performing operations on a Tensor level is often at the heart of the efficiency gains that can be achieved in practice.
* **Automatic Differentiation:** PyTorch’s autograd module enables automatic computation of gradients, which is essential for training neural networks and Gaussian processes. This feature allows you to easily backpropagate errors and update model parameters.
* **Dynamic Computational Graphs:** PyTorch uses dynamic computational graphs (also known as define-by-run), which are constructed on-the-fly during each iteration. This flexibility allows for more complex model architectures and easier debugging.

## 0.2 Code Examples

For those of you that are unfamiliar with PyTorch, it may be useful to read through the following code blocks for a quick introduction to how Tensor objects are defined and how automatic differentiation (AD) can be executed by calling [```.backward()```](https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html) on a Tensor that triggers differentiation to be carried out backward through the computational graph using chain rule.

### Tensor Manipulation

Let's start by running some basic Tensor operations.

In [ ]:
# fix the random seed
torch.manual_seed(42)

# define a 2d tensor (matrix) of 3x3 elements by drawing random numbers from a N(0,1)
a = torch.randn((3,3))

# perform element-wise addition with a number
b = a + 1

# perform matrix multiplication a * a.T (tranpose)
c = a @ a.T

# print the results
print("a is")
print(a)
print("b is")
print(b)
print("c is")
print(c)

### Automatic Differentiation

PyTorch’s ```autograd``` allows for automatic differentiation, which is crucial for optimizing models during training. Below is a simple example of how we can use this feature to compute the partial derivatives of an output $z = x \cdot y + y^2$ with respect to its two inputs $x$ and $y$ at specific values $x=2$ and $y=3$, i.e., $\left. \frac{\partial z}{\partial x} \right|_{x=2,y=3}$ and $\left. \frac{\partial z}{\partial y} \right|_{x=2,y=3}$.

In [ ]:
# create tensors with requires_grad=True to track computation
# (otherwise the variable is not tracked in the gradient calculation)
x = torch.tensor(2.0, requires_grad=True)
y = torch.tensor(3.0, requires_grad=True)

# define simple function
z = x * y + y**2

# compute gradients
z.backward()

# print results
print(f"dz/dx at x={x.item()} and y={y.item()}:  {x.grad.item()}")  # Should be y (3.0)
print(f"dz/dy at x={x.item()} and y={y.item()}:  {y.grad.item()}")  # Should be x + 2*y (2.0 + 6.0 = 8.0)

A special feature of PyTorch is its ability to easily handle "batching". Batching refers to processing multiple data points simultaneously rather than one at a time, and is essential for efficient computation. It is especially valuable when working with large datasets and neural networks. In the context of the automatic differentiation code above, batching would implying that we are computing the gradient for multiple $(x,y)$ points of interest at the same time. Let us see how this works:

In [ ]:
# create tensors with requires_grad=True to track computation
# (otherwise the variable is not tracked in the gradient calculation)
x = torch.tensor([2.0, 4.0, 6.0], requires_grad=True)  # batch of x values
y = torch.tensor([3.0, 5.0, 7.0], requires_grad=True)  # batch of y values

# define simple function
z = x * y + y**2

# compute gradients
#   .backward() only works natively on a scalar function, so we need to pass an
#   additional tensor to specify how to combine these elements into a single
#   scalar value to compute the gradients --> torch.ones_like(z) effectively
#   equally sums up the elements in the batch before calling .backward()
z.backward(torch.ones_like(z))

# print results
print(f"dz/dx at batch of (x,y) values:  {x.grad.detach().numpy()}")
print(f"dz/dy at batch of (x,y) values:  {y.grad.detach().numpy()}")

# 1. GPs from Scratch

In the presentation slides, we saw that a Gaussian process (GP) is a collection of random variables, any finite number of which have a joint Gaussian distribution. GPs are typically denoted by

$$
f(\mathbf{x}) \sim GP(\mu(\mathbf{x}), k(\mathbf{x}, \mathbf{x}'))
$$

where $\mu(\mathbf{x})$ is the mean function and $k(\mathbf{x}, \mathbf{x}')$ is the kernel function (equal to the covariance of the function values $f(\mathbf{x})$ and $f(\mathbf{x}')$ at the two points $\mathbf{x}$ and $\mathbf{x}'$). We will mostly assume the mean function is zero, i.e., $\mu(\mathbf{x} = 0)$ for convenience.

In the sections below, we want to build our own simple implementation of a GP with a radial basis function (RBF) (also known as squared exponential (SE)) kernel using standard PyTorch operations. These exercises are meant to provide more insight into the inner workings of a GP. Specifically, we will perform the following tasks:
* 1.1: Build our own implementation of the RBF kernel in PyTorch
* 1.2: Draw independent random samples from a GP prior
* 1.3: Using the Gaussian condtional formula to draw independent random samples from a GP posterior
* 1.4: Use the maginal log likelihod (MLL) to identify good kernel hyperparameters that are able to match observed data.

## 1.1 RBF Kernel

Recall that the RBF kernel has the following form

$$
k_\text{RBF}(\mathbf{x}, \mathbf{x}') = a^2 \exp\left(-\frac{\| \mathbf{x} - \mathbf{x}' \|^2}{2 l^2} \right)
$$

where $a$ is a hyperparameter related to the scale of the function, $l$ is the lengthscale that controls the correlation length (note we have assumed a single, common lengthscale for all dimensions for simplicity), and $\| \cdot \|$ denotes the standard Euclidean norm.

### Exercise

**TASK:** Your goal is to implement a Python function that evaluates the RBF kernel between two sets of points. The structure of the function (with all inputs and outputs) is provided in the code block below -- your goal is to complete the function. Note that your function needs to support batching, so I recommend that you use the [```torch.cdist```](https://pytorch.org/docs/stable/generated/torch.cdist.html) function to compute $\| \mathbf{x}_1 - \mathbf{x}_2 \|$ for a batch of inputs.

In [ ]:
# define the RBF kernel function
def rbf_kernel(x1, x2, l=1.0, a=1.0):
    """
    Computes the RBF kernel between two sets of points.

    Parameters
    ----------
    x1 : torch.Tensor
        First set of points.
    x2 : torch.Tensor
        Second set of points.
    l : float
        Lengthscale parameter.
    a : float
        Scale parameter.

    Returns
    -------
    torch.Tensor
        Kernel matrix.
    """
    ### FILL IN

# define hyperparameters
l = 1.0
a = 1.0

# create a grid of points in 1d
x = torch.linspace(-2, 2, 101)

# create a meshgrid for plotting purposes
X, Y = torch.meshgrid(x, x)

# evaluate covariance function between all elements of x
with torch.no_grad():
  K = rbf_kernel(x, x, l, a)
K = K.detach().numpy()

# plot covariance between x vector
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
c = ax.pcolor(X, Y, K, cmap=cm.Blues)
ax.invert_xaxis()
ax.set_title(f'a = {a} and l = {l}')
fig.colorbar(c, ax=ax)
plt.rcParams.update({'font.size':14});

### Solution

In [ ]:
# define the RBF kernel function
def rbf_kernel(x1, x2, l=1.0, a=1.0):
    """
    Computes the RBF kernel between two sets of points.

    Parameters
    ----------
    x1 : torch.Tensor
        First set of points.
    x2 : torch.Tensor
        Second set of points.
    l : float
        Lengthscale parameter.
    a : float
        Scale parameter.

    Returns
    -------
    torch.Tensor
        Kernel matrix.
    """
    # convert to 2d tensors if only 1d provided
    if len(x1.shape) == 1:
        x1 = x1.unsqueeze(-1)
    if len(x2.shape) == 1:
        x2 = x2.unsqueeze(-1)

    # compute the squared Euclidean distance between the points
    sqdist = torch.cdist(x1, x2)**2

    # compute the RBF kernel
    return a**2 * torch.exp(-sqdist / (2 * l**2))

# define hyperparameters
l = 1.0
a = 1.0

# create a grid of points in 1d
x = torch.linspace(-2, 2, 101)

# create a meshgrid for plotting purposes
X, Y = torch.meshgrid(x, x)

# evaluate covariance function between all elements of x
with torch.no_grad():
  K = rbf_kernel(x, x, l, a)
K = K.detach().numpy()

# plot covariance between x vector
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
c = ax.pcolor(X, Y, K, cmap=cm.Blues)
ax.invert_xaxis()
ax.set_title(f'a = {a} and l = {l}')
fig.colorbar(c, ax=ax)
plt.rcParams.update({'font.size':14});

## 1.2 GP Prior Samples

Now that we have the ability to evaluate the RBF kernel over a batch of input values, we want to use this capability to generate random sample GP paths. Recall from the presentation slides that we can always apply the whitening transformation to generate samples from a multivariate normal distribution. In other words, the following distributions are equivalent

$$
\mathbf{f} \sim N(\mathbf{m}, \mathbf{K})~~~ <=> ~~~ \mathbf{f} = \mathbf{m} + \mathbf{L}\mathbf{Z},
$$

where $\mathbf{K} = \mathbf{L}\mathbf{L}^\top$ such that $\mathbf{L}$ is the lower triangular Cholesky factor of the covariance matrix $\mathbf{K}$ and $\mathbf{Z} \sim N(0, I)$ is a set of independent standard normal random variables. Using this idea, generate 3 independent samples of a GP with zero mean and an RBF kernel with scale $a=1$ and lengthscale $l=1$ over a range of $x \in [-10, 10]$. Note that you can use the following two functions from the torch library:
* [```torch.linalg.cholesky(A)```](https://pytorch.org/docs/stable/generated/torch.linalg.cholesky.html) to compute the Cholesky decomposition of a matrix $A$.
* [```torch.randn((n,m))```](https://pytorch.org/docs/stable/generated/torch.randn.html) to generate an $n$ by $m$ matrix of standard normal random variables.

### Exercise

**TASK:** Fill in the missing parts of the code block below to generate a plot that shows (i) the 3 independent samples from the GP prior and (ii) the 95\% confidence region as a gray shaded region. Note that we can use the [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html) package to calculate the scaling factor to compute the lower and upper confidence region for a normal random variable, i.e., $Y \sim N(\mu_Y, \sigma_Y^2)$ will lie in the region

$$[\mu_Y + \sigma_Y\Phi^{-1}(0.025), \mu_Y + \sigma_Y\Phi^{-1}(0.975)]$$

with 95% probability where $\Phi^{-1}(p)$ is the inverse CDF for a standard normal random variable evaluated at probability level $p$.
* NOTE: You may run into issues with the Cholesky decomposition, as it can only be run on positive definite matrices and there can be numerical issues with the matrix is close to singular. A simple trick to fix this issue is to add some "jitter" (a small constant) to the diagonal. I recommend a value of ```1e-5*torch.eye(n)``` as a default jitter, though this can be increased slightly if needed.

In [ ]:
# fix the random seed
torch.manual_seed(42)

# create a list of points from [-10,10]
n = 101
x = torch.linspace(-10, 10, n)

# evaluate the rbf kernel at x for the specified hyperparamter values
### FILL IN

# calculate the Cholesky decomposition
### FILL IN

# create a plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# add the 95% confidence region to plot
sigma = K.diag().sqrt()
lq = sigma*stats.norm.ppf(0.025)
uq = sigma*stats.norm.ppf(0.975)
plt.plot(x, torch.zeros(n), '-b', linewidth=3)
plt.fill_between(x, lq, uq, color='gray', alpha=0.25)

# add three sample realizations to plot
### FILL IN

# set limits, labels, and font size
plt.xlim([-10,10])
plt.ylim([-4,4])
plt.xlabel('input, x')
plt.ylabel('output, f(x)')
plt.rcParams.update({'font.size':14});

### Solution

In [ ]:
# fix the random seed
torch.manual_seed(42)

# create a list of points from [-10,10]
n = 101
x = torch.linspace(-10, 10, n)

# evaluate the rbf kernel at x for the specified hyperparamter values
with torch.no_grad(): # add no_grad decorator since we do not need the gradient
    K = rbf_kernel(x, x, l=1, a=1)

# calculate the Cholesky decomposition
L = torch.linalg.cholesky(K + 1e-5*torch.eye(n))

# create a plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# add the 95% confidence region to plot
sigma = K.diag().sqrt()
lq = sigma*stats.norm.ppf(0.025)
uq = sigma*stats.norm.ppf(0.975)
plt.plot(x, torch.zeros(n), '-b', linewidth=3)
plt.fill_between(x, lq, uq, color='gray', alpha=0.25)

# add three sample realizations to plot
for i in range(3):
  f = torch.matmul(L, torch.randn((n,1)))
  plt.plot(x.detach().numpy(), f.detach().numpy(), '-', linewidth=1)

# set limits, labels, and font size
plt.xlim([-10,10])
plt.ylim([-4,4])
plt.xlabel('input, x')
plt.ylabel('output, f(x)')
plt.rcParams.update({'font.size':14});

## 1.3 GP Posterior Samples

We now want to test out our ability to generate samples from the posterior GP conditioned on observations. We will assume the observations do not have any noise. Therefore, the main result we need to use is the conditional Gaussian formula, which can be summarized as

$$
\begin{bmatrix}
X \\
Y
\end{bmatrix} \sim N\left( \begin{bmatrix} a \\ b \end{bmatrix}, \begin{bmatrix} A &C \\ C^\top &B \end{bmatrix} \right)
$$

leads to

$$
X | (Y=y) = N( a + C B^{-1} (y - b), A - C B^{-1} C^\top )
$$

For the GP posterior case, the variables $X$ correspond to the function values at the test locations $f(x_{1*}), \ldots, f(x_{t*})$ and the variables $Y$ correspond to the function values at the training locations $f(x_1), \ldots, f(x_n)$ for $t$ testing points and $n$ training points.

First, let us assume the following training points exist:

$$
x_1 = -6, ~~~ y_1 = 2, \\
x_2 = -4, ~~~ y_2 = 1, \\
~~x_3= 3, ~~~~~ y_3 = -1, \\
$$

and that the data is generated from a GP with RBF kernel under hyperparameters $l=1$ and $a=1$.

### Exercise

**TASK:** You want to calculate the GP posterior by generating the mean and covariance matrix for the vector of testing and training values $( f(x_{1*}), \ldots, f(x_{t*}), f(x_1), f(x_2), f(x_3) )$ and running the conditional formula above -- the test points correspond to a grid of $x$ values between -10 to 10 as before. Since you need all the different blocks (covariance between test-test, test-train, and train-train), I recommend you run these calculations separately to get each of the sub-blocks of the overall covariance matrix. Fill in the missing parts of the code block below to generate a plot that shows (i) the 3 independent samples from the GP posterior and (ii) the 95\% confidence region as a gray shaded region.
* NOTE: You can use ```torch.inverse(B)``` to compute the inverse of matrix $B$ for simplicity since we have a small training set; however, in general, you would want to use more efficient strategies that do not require explicitly computing and storing the inverse.

In [ ]:
# fix the random seed
torch.manual_seed(42)

# create a list of 101 test points from [-10,10]
n = 101
x_test = torch.linspace(-10, 10, n)

# create training data given in problem statement
### FILL IN

# need to evaluate the rbf kernel between different train, test combinations
### FILL IN

# now we can run the conditional calculation since means are 0 by assumption
### FILL IN

# we can then generate samples using the same whitening transformation idea as before
# but now we also need to track the mean function
### FILL IN

# create a plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# add the 95% confidence region, where we now need to account for the mean
### FILL IN
### Make sure to correctly compute mu, lq, and uq
plt.plot(x, mu, '-b', linewidth=3)
plt.fill_between(x, lq, uq, color='gray', alpha=0.25)

# add three sample realizations
### FILL IN

# set limits, labels, and font size
plt.xlim([-10,10])
plt.ylim([-4,4])
plt.xlabel('input, x')
plt.ylabel('output, f(x)')
plt.rcParams.update({'font.size':14});

### Solution

In [ ]:
# fix the random seed
torch.manual_seed(42)

# create a list of 101 test points from [-10,10]
n = 101
x_test = torch.linspace(-10, 10, n)

# create training data given in problem statement
x_train = torch.tensor([-6.0, -4.0, 3.0])
y_train = torch.tensor([2.0, 1.0, -1.0])

# need to evaluate the rbf kernel between different train, test combinations
a, l = 1, 1
K_test_test = rbf_kernel(x_test, x_test, l=l, a=a)
K_train_test = rbf_kernel(x_train, x_test, l=l, a=a)
K_train_train = rbf_kernel(x_train, x_train, l=l, a=a)

# now we can run the conditional calculation since means are 0 by assumption
Kinv_train_train = torch.inverse(K_train_train)
posterior_mean = K_train_test.T @ Kinv_train_train @ y_train
posterior_cov = K_test_test - K_train_test.T @ Kinv_train_train @ K_train_test

# we can then generate samples using the same whitening transformation idea as before
# but now we also need to track the mean function
mu = posterior_mean
L = torch.linalg.cholesky(posterior_cov + 1e-5*torch.eye(n))

# create a plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# add the 95% confidence region, where we now need to account for the mean
sigma = posterior_cov.diag().sqrt()
lq = mu + sigma*stats.norm.ppf(0.025)
uq = mu + sigma*stats.norm.ppf(0.975)
plt.plot(x_test, mu, '-b', linewidth=3)
plt.fill_between(x_test, lq, uq, color='gray', alpha=0.25)

# add three sample realizations
for i in range(3):
  f = mu.reshape((-1,1)) + torch.matmul(L, torch.randn((n,1)))
  plt.plot(x_test.detach().numpy(), f.detach().numpy(), '-', linewidth=1)

# set limits, labels, and font size
plt.xlim([-10,10])
plt.ylim([-4,4])
plt.xlabel('input, x')
plt.ylabel('output, f(x)')
plt.rcParams.update({'font.size':14});

## 1.4 Find the Hyperparameters

In practice, we often do not know the kernel hyperparameters $a$ and $l$ such that they must be learned/tuned given data. As discussed in the presentation slides, this tuning process can be done using the marginal log likelihood (MLL). For a GP prior $\mathbf{f} \sim N(0, \mathbf{K}_\theta)$ (evaluated at the training data points $\mathbf{X}$) with hyperparameters $\theta = (a,l)$ and a Gaussian likelihood with independent noise realizations $\mathbf{y} \mid \mathbf{f} \sim N(\mathbf{f}, \sigma_n^2 I)$ (with $\sigma_n^2$ denoting the noise variance), we can derive the following exact expression for the MLL:

$$
\log p( \mathbf{y} | \mathbf{X}, \theta, \sigma_n^2 ) = -\frac{1}{2} \mathbf{y}^\top (\mathbf{K}_\theta + \sigma_n^2 I )^{-1} \mathbf{y} - \frac{1}{2} \log\det( \mathbf{K}_\theta + \sigma_n^2 I ) - \frac{n}{2}\log(2\pi)
$$

where $n$ denotes the number of training datapoints. Although we could learn $\sigma_n^2$ along with $\theta$, here we will assume that the observation noise is fixed and small $\sigma_n^2 = 1 \times 10^{-4}$, meaning it effectively acts like a "jitter" in the covariance matrix.

Typically, one select the "optimal" hyperparameters as the ones that maximize the MLL, i.e., $\theta^* = \text{argmax}_{\theta} \log p( \mathbf{y} | \mathbf{X}, \theta, \sigma_n^2 )$ for a particular training dataset $\{ \mathbf{y}, \mathbf{X} \}$. Here, we want to do a similar procedure but not rigirously -- just a trial-and-error evaluation of the MLL to see if we can find settings that look like they match the "real" data generating process.

For simplicity, we have packaged the GP prior and posterior calculations that we wrote previously into helper functions that can be obtained by running the hidden code block below. The next code block shows how these functions can be used.

In [ ]:
# @title Helper Functions
# create helper function that can evaluate and plot prior predictive equations for GP
def GP_prior_predictive(x_test, l, a, sigma2_noise=1e-5, n_samples=3, plot_on=True):
    # evaluate the rbf kernel at x for the specified hyperparamter values
    with torch.no_grad(): # add no_grad decorator since we do not need the gradient
        K = rbf_kernel(x_test, x_test, l=l, a=a)

    # prior mean at the text locations
    n = x_test.shape[0]
    prior_mean = torch.zeros((n,1))

    # calculate the Cholesky decomposition
    mu = prior_mean
    L = torch.linalg.cholesky(K + sigma2_noise*torch.eye(n))

    # calculate samples
    gp_prior_samples = torch.zeros((n,n_samples))
    for i in range(n_samples):
        gp_prior_samples[:,i] = torch.matmul(L, torch.randn((n,1))).squeeze()

    if plot_on:
        # create a plot
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))

        # add the 95% confidence region to plot
        sigma = K.diag().sqrt()
        lq = sigma*stats.norm.ppf(0.025)
        uq = sigma*stats.norm.ppf(0.975)
        plt.plot(x_test, torch.zeros(n), '-b', linewidth=3)
        plt.fill_between(x_test, lq, uq, color='gray', alpha=0.25)

        # add three sample realizations to plot
        for i in range(n_samples):
            plt.plot(x_test.detach().numpy(), gp_prior_samples[:,i].detach().numpy(), '-', linewidth=1)

        # set limits, labels, and font size
        plt.xlim([-10,10])
        plt.ylim([-4,4])
        plt.xlabel('input, x')
        plt.ylabel('output, f(x)')
        plt.rcParams.update({'font.size':14});

    # return mean, covariance, and samples
    return prior_mean, K, gp_prior_samples

# create helper function that can evaluate and plot posterior predictive equations for GP
def GP_posterior_predictive(x_test, x_train, y_train, l, a, sigma2_noise=1e-5, n_samples=3, plot_on=True):
    # need to evaluate the rbf kernel between different train, test combinations
    with torch.no_grad(): # add no_grad decorator since we do not need the gradient
        K_test_test = rbf_kernel(x_test, x_test, l=l, a=a)
        K_train_test = rbf_kernel(x_train, x_test, l=l, a=a)
        K_train_train = rbf_kernel(x_train, x_train, l=l, a=a)

    # now we can run the conditional calculation since means are 0 by assumption
    L = torch.linalg.cholesky(K_train_train)  # K_train_train = L * L.T
    alpha = torch.cholesky_solve(y_train.unsqueeze(-1), L)  # Solves L * L.T * alpha = y_train
    posterior_mean = K_train_test.T @ alpha
    v = torch.cholesky_solve(K_train_test, L)  # Solves L * v = K_train_test
    posterior_cov = K_test_test - K_train_test.T @ v

    # calculate cholesky factor for test data
    n = x_test.shape[0]
    mu = posterior_mean.squeeze()
    L = torch.linalg.cholesky(posterior_cov + sigma2_noise*torch.eye(n))

    # calculate samples from posterior
    gp_posterior_samples = torch.zeros((n,n_samples))
    for i in range(n_samples):
        gp_posterior_samples[:,i] = (mu.reshape((-1,1)) + torch.matmul(L, torch.randn((n,1)))).squeeze()

    if plot_on:
        # create a plot
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))

        # add the 95% confidence region, where we now need to account for the mean
        sigma = torch.max(posterior_cov.diag(),torch.tensor([0.])).sqrt()
        lq = mu + sigma*stats.norm.ppf(0.025)
        uq = mu + sigma*stats.norm.ppf(0.975)
        plt.plot(x_test, mu, '-b', linewidth=3)
        plt.fill_between(x_test, lq, uq, color='gray', alpha=0.25)

        # add three sample realizations
        for i in range(n_samples):
            plt.plot(x_test.detach().numpy(), gp_posterior_samples[:,i].detach().numpy(), '-', linewidth=1)

        # add training points to the plot
        plt.scatter(x_train, y_train, s=200, color='k', marker="+", zorder=5)

        # set limits, labels, and font size
        plt.xlim([-10,10])
        factor = torch.ceil(torch.max(torch.abs(torch.min(lq)), torch.max(uq)))
        plt.ylim([-factor-2,factor+2])
        plt.xlabel('input, x')
        plt.ylabel('output, f(x)')
        plt.rcParams.update({'font.size':14});

    # return the mean and covariance
    return posterior_mean, posterior_cov, gp_posterior_samples

In [ ]:
# test and train points
x_test = torch.linspace(-10, 10, 101)
x_train = torch.tensor([-6.0, -4.0, 3.0])
y_train = torch.tensor([2.0, 1.0, -1.0])

# run operations on prior
torch.manual_seed(42)
prior_mean, prior_cov, prior_sample_paths = GP_prior_predictive(x_test, l=1, a=1, sigma2_noise=1e-5, n_samples=3, plot_on=True);

# run operations on posterior
torch.manual_seed(42)
post_mean, post_cov, post_sample_paths = GP_posterior_predictive(x_test, x_train, y_train, l=1, a=1, sigma2_noise=1e-5, n_samples=3, plot_on=True);

### Exercise

**TASK:** Fill in the missing parts of the code block below to evaluate the MLL matching the assumptions listed above (GP prior with zero mean and RBF kernel) for specific hyperparamter $\theta$ values. The data is pre-loaded below. To help, we have packaged the code that we wrote in sections 1.2 and 1.3 into helper functions that you can create by running the hidden code block above.

In [ ]:
# observed data
x_train = torch.Tensor([-8.0000, -6.2222, -4.4444, -2.6667, -0.8889,  0.8889,  2.6667,  4.4444,   6.2222,  8.0000])
y_train = torch.Tensor([-1.1544, -0.9582,  0.3840,  1.0536,  0.2901, -0.6560, -0.9193, -0.2800,   0.3737,  0.5213])

# create a function that evaluates the MLL given y and K
### FILL IN

# create test points at which to evaluate the
x_test = torch.linspace(-10, 10, 101)

# create a list of possible lengthscale values
l_list = torch.linspace(0.2, 4, 101)

# evaluate the mll for all values in this list
a_true = 1
sigma2_noise = 1e-4
mll_list = torch.zeros(len(l_list))
for i in range(len(l_list)):
    ### FILL IN
    ### Evaluate the MLL at the different lengthscales in l_list
    mll_list[i] =

# create a plot of lengthscale values vs MLL
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plt.plot(l_list, mll_list, '-b', linewidth=3)
plt.xlabel('lengthscale, l')
plt.ylabel('MLL')
plt.rcParams.update({'font.size':14})

### Solution

In [ ]:
# observed data
x_train = torch.Tensor([-8.0000, -6.2222, -4.4444, -2.6667, -0.8889,  0.8889,  2.6667,  4.4444,   6.2222,  8.0000])
y_train = torch.Tensor([-1.1544, -0.9582,  0.3840,  1.0536,  0.2901, -0.6560, -0.9193, -0.2800,   0.3737,  0.5213])

# create a function that evaluates the MLL given y and K
def GP_MLL(y, K, sigma2_noise=1e-4):
    n = y.shape[0]
    y = y.reshape((-1,1))
    Ky = K + sigma2_noise*torch.eye(n)
    mll = -0.5*y.T @ torch.inverse(Ky) @ y - 0.5*torch.logdet(Ky) - n/2*np.log(2*np.pi)
    return mll.item()

# create test points at which to evaluate the
x_test = torch.linspace(-10, 10, 101)

# create a list of possible lengthscale values
l_list = torch.linspace(0.2, 4, 101)

# evaluate the mll for all values in this list
a_true = 1
sigma2_noise = 1e-4
mll_list = torch.zeros(len(l_list))
for i in range(len(l_list)):
    mu_prior, K_prior, sample_paths_prior = GP_prior_predictive(x_train, l=l_list[i], a=a_true, sigma2_noise=sigma2_noise, n_samples=0, plot_on=False);
    mll_list[i] = GP_MLL(y_train, K_prior, sigma2_noise=sigma2_noise)

# create a plot of lengthscale values vs MLL
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plt.plot(l_list, mll_list, '-b', linewidth=3)
plt.xlabel('lengthscale, l')
plt.ylabel('MLL')
plt.rcParams.update({'font.size':14})
### TRUE ANSWER: l = 2.5

You can see that the optimal value that maximizes the MLL is around 2.4 to 2.6, which is close to the real value of 2.5. Note that this will not always be the case, as it depends on the details of the dataset and the structure of the model. As seen in the slides, it is possible to overfit a model when maximizing the MLL, especially when you have a large number of hyperparameters that are being estimated.

# 2. Introduction to GPyTorch

GPyTorch is a Python library built on top of PyTorch that provides a scalable and flexible framework for Gaussian Processes (GPs). Gaussian Processes are a powerful non-parametric method for regression, classification, and other machine learning tasks, particularly useful for modeling uncertainty and making predictions with probabilistic interpretations. It leverages the computational efficiency of PyTorch, allowing for the easy integration of Gaussian Processes into deep learning workflows. It is especially well-suited for tasks that require scalable GP models, such as working with large datasets or building complex GP models like deep kernel learning.

The sections below are meant to introduce you how to use core components of GPyTorch. Specifically, we will:
* 2.1: See how to define a model and likelihood function.
* 2.2: Learn how to train a model using torch optimizers.
* 2.3: See how to make predictions with a trained model (perform inference).
* 2.4: Create a custom kernel object.
<!-- * 2.4: Use the default GP training approach in BoTorch as an alternative.  -->

Before getting started, make sure you have GPyTorch and BoTorch installed (under the Setup tab).

## 2.1 Defining a Model

The basic features of a model are summarized in the documentation [here](https://docs.gpytorch.ai/en/v1.6.0/examples/01_Exact_GPs/Simple_GP_Regression.html). The design philosophy behind GPyTorch is that a user is provided with the tools necessary to quickly construct a GP model (as opposed to coming with a pre-packaged model). The rationale is that it is often important for a user to have flexibility to include whatever components are necessary.

For most GP regression models, you will need to construct the following GPyTorch objects:
* A **GP Model** (```gpytorch.models.ExactGP```) - This handles most of the inference.
* A **Likelihood** function (```gpytorch.likelihoods.GaussianLikelihood```) - This is the most common likelihood used for GP regression.
* A **Mean** function - Defines the prior mean of the GP. (A ```gpytorch.means.ConstantMean()``` is usually a good place to start.)
* A **Kernel** function - Defines the prior covariance of the GP. (e.g., ```gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())``` provides the RBF kernel with a constant scaling factor).
* A **MultivariateNormal** Distribution (```gpytorch.distributions.MultivariateNormal```) - This is the object used to represent multivariate normal distributions.

To define a model, we typically start by subclassing an existing "base" model in GPyTorch. You can find a complete list of models [here](https://docs.gpytorch.ai/en/stable/models.html). For our purposes, we will focus on the exact inference procedure. The main things we need to specify are then:
* An **```__init__```** method that takes the training data and a likelihood, and constructs whatever objects are necessary for the model’s forward method. This will most commonly include things like a mean module and a kernel module.
* A **```forward```** method that takes in $n \times d$ data ```x``` and returns a ```MultivariateNormal``` with the prior mean and covariance evaluated at ```x```. In other words, we return the vector $\mu(x)$ and the $n\times n$ matrix $K_{xx}$ representing the prior mean and covariance matrix of the GP.

In [ ]:
# define exact GP model (called MyGP) with constant mean and RBF kernel
class MyGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MyGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

To actually create a model object, we need to pass in some training data. Let's generate a simple set of data from a sin function but include some degree of Gaussian noise in the observations. We will specify a Gaussian likelihood as well.

In [ ]:
# generate a set of training points
torch.manual_seed(42)
train_x = torch.linspace(0, 1, 50)
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.1

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MyGP(train_x, train_y, likelihood)

### Hyperparameters

Whenever you create a model, it is likely to have some parameters attached to it where "parameters" refer to the objects of type ```torch.nn.Parameter``` that have gradients automatically filled in by the autograd functionality discussed previously in this notebook. In the context of GP regression, these are the hyperparameters of the kernel and/or likelihood function. We can explicitly look at the full set of parameters attached to a model using the ```model.named_parameters()``` generator, as shown in the following code block.

In [ ]:
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.item()}')

Notice that the parameters are referencing "raw" values such as raw_noise, raw_outputscale, and raw_lengthscale. This is because, in GPyTorch, parameters can be constrained (e.g., all of the previous parameters referenced must be positive) such that it uses "constraint functions" to enforce these constraints directly. This leads to a critical difference between **raw parameters** and **actual parameters**. For example, we can extract the raw_outputscale and its associated constraint and check that, when it is run through the transform and then the inverse transform, it leads back to the raw value.

In [ ]:
raw_outputscale = model.covar_module.raw_outputscale
constraint = model.covar_module.raw_outputscale_constraint
print(f'Raw outputscale: {raw_outputscale.detach().item()}')
print(f'Actual (transformed) outputscale: {constraint.transform(raw_outputscale).detach().item()}')
print(f'Is inverse_transform(transform(raw_outputscale)) == raw_outputscale? {torch.equal(constraint.inverse_transform(constraint.transform(raw_outputscale)), raw_outputscale)}')

Since dealing with raw parameter values is often annoying, most of the built-in GPyTorch modules that define raw parameters define convenience getters and setters for dealing with transformed values directly.


In [ ]:
# recreate model to reset outputscale
model = MyGP(train_x, train_y, likelihood)

# convenient way of getting true outputscale
print(f'Actual outputscale: {model.covar_module.outputscale}')

# convenient way of setting true outputscale
model.covar_module.outputscale = 2.
print(f'Actual outputscale after setting: {model.covar_module.outputscale}')

### Model modes

Following the standard approach in PyTorch, the ExactGP (and other GPyTorch modules) has a ```.train()``` and ```.eval()``` mode.
* ```.train()```: This mode should be used when optimizing the hyperparameters of the model.
* ```.eval()```: This mode should be used for computing predictions from the model posterior.

## 2.2 Training a Model

Below, we show how to learn/train the hyperparameters a GP using the "Type-II maximum likelihood estimator", which corresponds to maximizing the marginal log likelihood function. Just like in PyTorch, the core training loop is written by the user. This means we can use all of the available optimization algorithms from ```torch.optim```. Additionally, all of the trainable parameters of the model should be of type ```torch.nn.Parameter``` and should be attached to the named parameters in the model (as shown in a previous cell).

### Adam

The boilerplate code often works quite well, which has the same basic components as the standard PyTorch training loop:
1. Zero all parameter gradients
2. Call the model and compute the loss
3. Call backward on the loss to fill in gradients (autograd)
4. Take a step on the optimizer
Let's see how this works below for our sin example using the [Adam](https://arxiv.org/pdf/1412.6980) optimizer.

In [ ]:
# recreate model and likelihood to reset hyperparameters
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MyGP(train_x, train_y, likelihood)

# set the model into train mode
model.train()

# "loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# loop for some number training iterations
training_iter = 50
for i in range(training_iter):
    # zero gradients from previous iteration
    optimizer.zero_grad()
    # calc output from model
    output = model(train_x)
    # calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    # print the current loss
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    # take a step in the optimizer
    optimizer.step()

### L-BFGS

Just to illustrate the flexibility of the optimization process, we also show how you can replace the first-order Adam method with a quasi-Newton method such as the [L-BFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS) optimization algorithm. L-BFGS creates an estimate of the inverse hessian matrix for the objective function using a history of past updates. Due to its more complicated structure, the training loop does need to be modified some as shown in the code block below. However, notice how we can easily make such modifications by building on all the great tooling developed over many years in PyTorch.

In [ ]:
# recreate model and likelihood to reset hyperparameters
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MyGP(train_x, train_y, likelihood)

# set the model into train mode
model.train()

# "loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# define optimizer -- now LBFGS
lbfgs = torch.optim.LBFGS(model.parameters(), history_size=20, max_iter=5, line_search_fn="strong_wolfe")

# define a closure needed for LBFGS
def closure():
    lbfgs.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    return loss

# loop for some number training iterations
training_iter = 10
for i in range(training_iter):
    # calculate the loss for monitoring
    with torch.no_grad():
        output = model(train_x)
        loss = -mll(output, train_y)
    # print the current loss
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    # take step in lbfgs using closure
    lbfgs.step(closure)

Notice that the best-found parameters are quite similar between Adam and L-BFGS; however, there are some minor differences. Technically, the loss is slightly better with the settings found by L-BFGS (-0.609 vs. -0.607). It is also interesting to note that L-BFGS converges to stable values in many fewer iterations, though the cost per iteration is a bit higher than that of Adam.

## 2.3 Making Model Predictions

To make predictions with the model, all we need to do is put the model and likelihood into ```.eval()``` mode and then call these modules on relevant test data. We perform those computations and then plot the resulting model fit in the cells below.

### Get Test Predictions

Just as a user-defined GP model returns a ```MultivariateNormal``` object with the prior mean and covariance from the ```forward``` method, a trained GP model put into ```.eval()``` mode returns a ```MultivariateNormal``` containing the posterior mean and covariance. Thus, we can get the predictive mean, variance, covariance, and draw samples from the GP at the test points using the calls shown in the following cell block:


In [ ]:
# get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# generate some test points
test_x = torch.linspace(0, 1, 101)

# evaluate the model to get f predictions and likelihood to get y predictions
#   The gpytorch.settings.fast_pred_var context is not needed, but yields faster predictive distributions using LOVE.
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    f_preds = model(test_x)
    y_preds = likelihood(model(test_x))

# extract the mean, variance, and covariance information from the posterior distribution
f_mean = f_preds.mean
f_var = f_preds.variance
f_covar = f_preds.covariance_matrix

# generate some random samples from the posterior
f_samples = f_preds.sample(sample_shape=torch.Size([10]))

### Plot the Model Fit

Below, we plot the mean and confidence region of the (posterior) GP model. We use the ```.confidence_region()``` method that automatically returns 2 standard deviations above and below the mean. Note that we plot the confidence region for both the function $f(x)$ and the observations of the function $y = f(x) + \epsilon$, with the latter being computed by combining the model predictions with the likelihood.


In [ ]:
# create a plot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
# get upper and lower confidence bounds
lower, upper = f_preds.confidence_region()
lower_obs, upper_obs = y_preds.confidence_region()
# Plot training data as black stars
ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
# Plot predictive means as blue line
ax.plot(test_x.numpy(), f_mean.numpy(), 'b')
# shade between the lower and upper confidence bounds for function f
ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.25)
# shade between the lower and upper confidence bounds for outputs y
ax.fill_between(test_x.numpy(), lower_obs.numpy(), upper_obs.numpy(), alpha=0.25)
ax.set_ylim([-2, 2])
ax.legend(['observed data', 'mean', 'confidence for f', 'confidence for y'])
plt.xlabel('input, x')
plt.ylabel('output, f(x)')
plt.rcParams.update({'font.size':14});

## 2.4 Custom Kernels

A powerful feature of GPyTorch is the ability to create custom kernel functions, as long as they can be implemented with torch functions (that allow for easy automatic differentiation). We will explore implementing the Brownian motion kernel that has the following form:

$$
k(x, x') = \sigma^2 \min(x, x')
$$

This kernel is particularly useful in finance (e.g., modeling stock prices), physics (e.g., particle trajectories), and other areas where the modeled process follows a Brownian motion. As such, it is a popular covariance function for GPs when modeling processes that exhibit continuous, non-stationary, and unpredictable paths over time ($x$ normally represents time in these applications).








### Exercise

**TASK:** Your goal is to implement this kernel (which does not come natively in GPyTorch) and apply it to a simulated Brownian motion trajectory. You will need to attach a hyperparameter that corresponds to the scale $\sigma^2$. To help you, I have provided several of the components needed in the code block below. This code is based on the [custom kernel tutorial](https://docs.gpytorch.ai/en/v1.6.0/examples/00_Basic_Usage/Implementing_a_custom_Kernel.html?highlight=custom%20kernel) provided in the GPyTorch documentation. You will see that I registered a parameter ```raw_scale``` that is the "raw" form of the parameter as well as a positive constraint ```raw_scale_constraint``` that can be used to transform from a raw to the actual parameter value. The main missing parts are related to the forward evaluation of the kernel. Note that this will require you to compute pairwise minimums across all the combinations of $x$ and $x'$ (fed in batchwise as $n \times 1$ matrices since $x$ must be 1d). You can do this using the standard ```torch.min()```, though you need to be careful with how call it.
* NOTE: You should look through all components of the code carefully to make sure you understand how the steps work.

In [ ]:
##############################################################
# Only modify this part of code block; replace FILL IN parts
##############################################################
# Create Brownian Motion Kernel
class BrownianMotionKernel(gpytorch.kernels.Kernel):
    # the brownian kernel is not stationary
    is_stationary = False

    # we will register the scale parameter when initializing the kernel
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # register the raw scale parameter
        self.register_parameter(name='raw_scale', parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 1, 1)))

        # set the parameter constraint to be positive
        scale_constraint = Positive()

        # register the constraint
        self.register_constraint("raw_scale", scale_constraint)

    # now set up the 'actual' paramter
    @property
    def scale(self):
        # when accessing the parameter, apply the constraint transform
        return self.raw_scale_constraint.transform(self.raw_scale)

    @scale.setter
    def scale(self, value):
        return self._set_scale(value)

    def _set_scale(self, value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_scale)
        # when setting the paramater, transform the actual value to a raw one by applying the inverse transform
        self.initialize(raw_scale=self.raw_scale_constraint.inverse_transform(value))

    # this is the kernel function
    def forward(self, x1, x2, **params):
        # check statements
        assert x1.shape[-1] == 1 and x2.shape[-1] == 1 # only works for 1d input
        assert torch.all(x1 >= 0) and torch.all(x2 >= 0) # negative times are not currently supported

        # compute the pairwise minimums
        ### FILL IN

        # make sure values are positive
        ### FILL IN

        # return the scaled covariance matrix
        ### FILL IN
##############################################################
##############################################################

# Generate training data with fixed seed
torch.manual_seed(42)
train_x = torch.linspace(0, 1, 100)
train_y = torch.cumsum(0.05*torch.randn(train_x.size()), dim=0)  # Simulate Brownian motion data

# Define GP model with Brownian motion kernel
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = BrownianMotionKernel()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Set up the likelihood and model [here we use a fixed noise model]
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise = 1e-4*torch.ones_like(train_x))
model = GPModel(train_x, train_y, likelihood)

# Train the model
model.train()

# Use the Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs is the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# Training loop
training_iter = 200
for i in range(training_iter):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()

# Convert model to eval mode to make predictions on test data
model.eval()

# Make predictions
test_x = torch.linspace(0, 1.6, 101)
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    f_preds = model(test_x)

# Make plot of predictions
with torch.no_grad():
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    # get upper and lower confidence bounds
    lower, upper = f_preds.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), f_preds.mean.numpy(), 'b')
    # shade between the lower and upper confidence bounds for function f
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.25)
    ax.legend(['observed data', 'mean', 'confidence'], loc='upper left')
    plt.xlabel('input, x')
    plt.ylabel('output, f(x)')
    plt.rcParams.update({'font.size':14});

### Solution

In [ ]:
# Create Brownian Motion Kernel
class BrownianMotionKernel(gpytorch.kernels.Kernel):
    # the brownian kernel is not stationary
    is_stationary = False

    # we will register the scale parameter when initializing the kernel
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # register the raw scale parameter
        self.register_parameter(name='raw_scale', parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 1, 1)))

        # set the parameter constraint to be positive
        scale_constraint = Positive()

        # register the constraint
        self.register_constraint("raw_scale", scale_constraint)

    # now set up the 'actual' paramter
    @property
    def scale(self):
        # when accessing the parameter, apply the constraint transform
        return self.raw_scale_constraint.transform(self.raw_scale)

    @scale.setter
    def scale(self, value):
        return self._set_scale(value)

    def _set_scale(self, value):
        if not torch.is_tensor(value):
            value = torch.as_tensor(value).to(self.raw_scale)
        # when setting the paramater, transform the actual value to a raw one by applying the inverse transform
        self.initialize(raw_scale=self.raw_scale_constraint.inverse_transform(value))

    # this is the kernel function
    def forward(self, x1, x2, **params):
        # check statements
        assert x1.shape[-1] == 1 and x2.shape[-1] == 1 # only works for 1d input
        assert torch.all(x1 >= 0) and torch.all(x2 >= 0) # negative times are not currently supported

        # compute the pairwise minimums
        covar_matrix = torch.min(x1, x2.T)

        # make sure values are positive
        covar_matrix.where(covar_matrix <= 0, torch.as_tensor(1e-6))

        # return the scaled covariance matrix
        return self.scale * covar_matrix

# Generate training data with fixed seed
torch.manual_seed(42)
train_x = torch.linspace(0, 1, 100)
train_y = torch.cumsum(0.05*torch.randn(train_x.size()), dim=0)  # Simulate Brownian motion data

# Define GP model with Brownian motion kernel
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = BrownianMotionKernel()

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Set up the likelihood and model [here we use a fixed noise model]
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise = 1e-4*torch.ones_like(train_x))
model = GPModel(train_x, train_y, likelihood)

# Train the model
model.train()

# Use the Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs is the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# Training loop
training_iter = 200
for i in range(training_iter):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    optimizer.step()

# Convert model to eval mode to make predictions on test data
model.eval()

# Make predictions
test_x = torch.linspace(0, 1.6, 101)
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    f_preds = model(test_x)

# Make plot of predictions
with torch.no_grad():
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    # get upper and lower confidence bounds
    lower, upper = f_preds.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x.numpy(), f_preds.mean.numpy(), 'b')
    # shade between the lower and upper confidence bounds for function f
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.25)
    ax.legend(['observed data', 'mean', 'confidence'], loc='upper left')
    plt.xlabel('input, x')
    plt.ylabel('output, f(x)')
    plt.rcParams.update({'font.size':14});

# 3. GP Regression: Modeling CO$_2$ Levels at the Mauna Loa Observatory

In this section, we want to use the well-known atmospheric CO$_2$ modeling problem to reinforce the different concepts we learned in the previous sections. The [data](https://cir.nii.ac.jp/crid/1571980076132477696) consists of monthly average values of the CO$_2$ concentration in parts per million by volume (ppmv). This data was collected at the Mauna Loa Observatory, Hawaii, between 1958 and 2003 (with some missing values). We want to model this data using a GP with a relatively complex kernel -- the need for a complex kernel will become apparent shortly.


## Exercise

**TASK:** The goal of this exercise is effectively to reproduce the sklearn implementation of this problem, which can be found [here](https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpr_co2.html). We will first prepare the data in a usable format. Then, you will attempt to train a GP model with a relatively simple kernel wherein you will see that the fit is not particularly good. By implementing the more complex kernel suggested in Section 5.4.3. of the [Gaussian Processes for Machine Learning](https://gaussianprocess.org/gpml/chapters/RW.pdf) book, we can achieve a much better fit.

### Prepare Train and Test Data

We first just load the data. We will use all available observations for training, which includes times from around 1958 to 2001.

In [ ]:
# create a function that fetches data and processes it as needed
def load_mauna_loa_atmospheric_co2():
    ml_data = fetch_openml(data_id=41187, as_frame=False)
    months = []
    ppmv_sums = []
    counts = []

    y = ml_data.data[:, 0]
    m = ml_data.data[:, 1]
    month_float = y + (m - 1) / 12
    ppmvs = ml_data.target

    for month, ppmv in zip(month_float, ppmvs):
        if not months or month != months[-1]:
            months.append(month)
            ppmv_sums.append(ppmv)
            counts.append(1)
        else:
            # aggregate monthly sum to produce average
            ppmv_sums[-1] += ppmv
            counts[-1] += 1

    months = np.array(months).reshape(-1, 1)
    avg_ppmvs = np.array(ppmv_sums) / counts
    return months, avg_ppmvs

# function to convert numpy arrays to torch tensors
def to_torch(V):
    return torch.tensor(V).flatten().double()

# call data loading function to get raw data
X_raw, Y_raw = load_mauna_loa_atmospheric_co2()

# split data into train and test sets and convert to torch
train_x = to_torch(X_raw)
train_y = to_torch(Y_raw)

# let's just make a simple plot, so we can visualize the data
f, ax = plt.subplots(1, 1, figsize=(10, 4))
ax.scatter(train_x, train_y, 5, 'b')
plt.xlabel('Year')
plt.ylabel('CO$_2$ Concentration (ppm)')
plt.grid(True)
plt.rcParams.update({'font.size':14})

We immediately notice some important things about this data:
* it has a long term rising trend;
* it has a pronounced seasonal variation;
* and it has some smaller short term irregularities.

Each of these types of behaviors can be captured with separate covariance functions (kernels) that can be combined together to deal with these individual properties.

### Train and Test GP Model

I recommend tackling the training and testing component in 2 separate parts.
1. First, just write the individual sections of the code (the GP model, the training loop, and the prediction plot) using a simple RBF kernel. You can then investigate the performance and see what MLL it achieves.
2. Second, you can then easily update your code developed in the first part by replacing the ```covar_module``` attached to the GP using the suggested approach in the [sklearn example](https://scikit-learn.org/stable/auto_examples/gaussian_process/plot_gpr_co2.html). Note that you need to stick very close to their suggested form including using the same initial guesses for the hyperparameters for your model to achieve good performance.
  * You can scroll down to the "Design the proper kernel" section to see the details of the kernel, which is the sum of 4 individual kernels. It has a form like: $k_\text{long-term}(x, x') + k_\text{seasonal}(x, x') + k_\text{irregularities}(x, x') + k_\text{noise}(x, x')$.
  * Compare the MLL you get with this more complex kernel to the standard RBF kernel, which one would you prefer?

TIPS: Here are some important tips you should follow when setting up the code in Step 1.
* You do not need to normalize the input or output values, as that will change the meaning of the hyperparameters (and you will need to derive updated settings in the scaled space). However, you do need to subtract the mean value from the output values before training, i.e., calculate ```y_mean = train_y.mean()``` and subtract it from ```train_y``` before passing to the GP model.
* The loss landscape is very complex and thus I found that most of the default optimizers failed. As such, I recommend using the ```fit_gpytorch_mll``` function from BoTorch, which was already loaded with the imports at the beginning of the notebook. All you need to do to use this function is to pass the evaluated the MLL evaluated from the model in ```.train()``` mode. It will automatically optimize the hyperparameters using a slightly more sophisticated version of the L-BFGS algorithm.
* You want to run your test predictions from 1958 to 2025 broken into 1001 increments.

In [ ]:
### put your solution here


Go back and modify the kernel and see if you can get better performance. I recommend trying the following kernels: (i) spectral mixture (need to be careful with initialization), (ii)

## Solution

In [ ]:
### This code block implements both the RBF and the Complex kernel.
### You can switch between them setting "use_complex" to True or False

# set flag for a complex kernel or not
#   False implies RBF while True implies the additive complex kernel
use_complex = True

# calculate mean of training outputs
y_mean = train_y.mean()

# define exact GP model
class GPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(GPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# define a function that generates the complex kernel
def complex_kernel():
    #### long term
    k_long_term_RBF = gpytorch.kernels.RBFKernel()
    k_long_term_RBF.lengthscale = 50.0

    k_long_term = gpytorch.kernels.ScaleKernel(k_long_term_RBF)
    k_long_term.outputscale = 50.0**2

    #### seasonal
    k_seasonal_RBF = gpytorch.kernels.RBFKernel()
    k_seasonal_RBF.lengthscale = 100.0

    k_seasonal_periodic = gpytorch.kernels.PeriodicKernel()
    k_seasonal_periodic.period_length = 1.0
    k_seasonal_periodic.lengthscale = 1.0

    k_seasonal = gpytorch.kernels.ScaleKernel(k_seasonal_RBF * k_seasonal_periodic)
    k_seasonal.outputscale = 4.0

    ### medium term
    k_medium_RQ = gpytorch.kernels.RQKernel()
    k_medium_RQ.alpha = 1.0
    k_medium_RQ.lengthscale = 1.0

    k_medium = gpytorch.kernels.ScaleKernel(k_medium_RQ)
    k_medium.outputscale = 0.5**2

    #### noise term
    k_noise_RBF = gpytorch.kernels.RBFKernel()
    k_noise_RBF.lengthscale = 0.1

    k_noise = gpytorch.kernels.ScaleKernel(k_noise_RBF)
    k_noise.outputscale = 0.1**2

    return k_long_term + k_seasonal + k_medium + k_noise

# get the kernel we will be using
if use_complex:
    kernel = complex_kernel()
else:
    kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

# define the model and likelihood function
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPModel(train_x, train_y - y_mean, likelihood, kernel)

# make sure the model is in train mode
model.train()

# get "loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# call the optimization wrapper in BoTorch for training GP models
fit_gpytorch_mll(mll)

# evaluate and print the marginal log likelihood
model.train()
with torch.no_grad():
    output = model(train_x)
    loss = -mll(output, train_y - y_mean)
    print('MLL for optimal hyperparameters: %.3f' % -loss.item())

# put model into eval mode to start making predictions
model.eval()

# make predictions
test_x = torch.linspace(1958, 2025, 1001)
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    f_preds = model(test_x)

# plot predictions
with torch.no_grad():
    # make plot
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    # get upper and lower confidence bounds
    lower, upper = f_preds.confidence_region()
    # add back mean value to get correct bounds
    lower, upper = lower + y_mean, upper + y_mean
    # plot training data as blue dots
    ax.scatter(train_x, train_y, 5, 'b')
    # plot predictive means as a gray line
    f_mean = f_preds.mean + y_mean
    ax.plot(test_x.numpy(), f_mean.numpy(), 'gray')
    # shade between the lower and upper confidence bounds for function f
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.25, color='gray')
    # add a legend and labels
    ax.legend(['observed data', 'mean', 'confidence'], loc='upper left')
    plt.xlabel('input, x')
    plt.ylabel('output, f(x)')
    plt.rcParams.update({'font.size':14});